In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
import pandas as pd

# Load the dataset
data = pd.read_csv('water_potability.csv')

# Calculate the count of missing values for each column
missing_counts = data.isnull().sum()

# Select the indices where the missing counts are zero
zero_missing_indices = missing_counts.index[missing_counts.values == 0]

# Select the indices where the missing counts are non-zero
non_zero_missing_indices = missing_counts.index[missing_counts.values > 0]

# Create a new dataset with only the columns having no missing values
data_filled = data[zero_missing_indices]

# Create a new dataset with only the columns having missing values
data_skipped = data[non_zero_missing_indices]

# Add the first column from data_skipped to data_filled
data1 = pd.concat([data_filled, data_skipped.iloc[:, 0:1]], axis=1)

# Select rows from data1 where the 'ph' column is not NaN
data1_filled = data1[data1['ph'].notna()]

# Split data into features and target
X = data1_filled.drop(['ph'], axis=1)
y = data1_filled['ph']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the search space for hyperparameters
search_space = {
    'num_layers': np.arange(1, 20),  # Number of layers
    'neurons_per_layer': np.arange(1, 128, 1),  # Neurons per layer
    'activation_functions': ['relu', 'sigmoid', 'tanh', 'softmax', 'softplus',
                             'softsign', 'elu', 'selu', 'gelu', 'hard_sigmoid', 'linear'],  # Activation functions
    'optimizers': ['adam', 'sgd', 'rmsprop', 'adagrad', 'adadelta', 'adamax', 'nadam'],  # Optimizers
    'alphas': [0.0001, 0.001, 0.01, 0.1],  # Learning rates
    'loss_functions': ['mean_squared_error', 'mean_absolute_error',  'logcosh', 'huber_loss']
}

# Function to get optimizer object
def get_optimizer(name, alpha):
    optimizers = {
        'adam': Adam(learning_rate=alpha),
        'sgd': SGD(learning_rate=alpha),
        'rmsprop': RMSprop(learning_rate=alpha),
        'adagrad': Adagrad(learning_rate=alpha),
        'adadelta': Adadelta(learning_rate=alpha),
        'adamax': Adamax(learning_rate=alpha),
        'nadam': Nadam(learning_rate=alpha)
    }
    return optimizers[name]

# Define the fitness function
def evaluate_model(params):
    num_layers = params['num_layers']
    neurons_per_layer = params['neurons_per_layer']
    activation_functions = params['activation_functions']
    optimizer_name = params['optimizer']
    alpha = params['alpha']
    loss_function = params['loss_function']
    
    model = Sequential()
    model.add(Dense(neurons_per_layer[0], activation=activation_functions[0], input_shape=(X_train.shape[1],)))
    for i in range(1, num_layers):
        model.add(Dense(neurons_per_layer[i], activation=activation_functions[i]))
    model.add(Dense(1))  # Output layer
    optimizer = get_optimizer(optimizer_name, alpha)
    model.compile(optimizer=optimizer, loss=loss_function)
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    return r2

# Define the genetic algorithm operations
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        num_layers = np.random.choice(search_space['num_layers'])
        neurons_per_layer = np.random.choice(search_space['neurons_per_layer'], num_layers).tolist()
        activation_functions = np.random.choice(search_space['activation_functions'], num_layers).tolist()
        optimizer = np.random.choice(search_space['optimizers'])
        alpha = np.random.choice(search_space['alphas'])
        loss_function = np.random.choice(search_space['loss_functions'])
        chromosome = {
            'num_layers': num_layers,
            'neurons_per_layer': neurons_per_layer,
            'activation_functions': activation_functions,
            'optimizer': optimizer,
            'alpha': alpha,
            'loss_function': loss_function
        }
        population.append(chromosome)
    return population

def crossover(parent1, parent2):
    child = {}
    child['num_layers'] = np.random.choice([parent1['num_layers'], parent2['num_layers']])
    child['neurons_per_layer'] = [np.random.choice([p1, p2]) for p1, p2 in zip(parent1['neurons_per_layer'], parent2['neurons_per_layer'])]
    child['activation_functions'] = [np.random.choice([p1, p2]) for p1, p2 in zip(parent1['activation_functions'], parent2['activation_functions'])]
    child['optimizer'] = np.random.choice([parent1['optimizer'], parent2['optimizer']])
    child['alpha'] = np.random.choice([parent1['alpha'], parent2['alpha']])
    child['loss_function'] = np.random.choice([parent1['loss_function'], parent2['loss_function']])
    # Adjust the lists to the number of layers
    child['neurons_per_layer'] = child['neurons_per_layer'][:child['num_layers']]
    child['activation_functions'] = child['activation_functions'][:child['num_layers']]
    return child

def mutate(chromosome):
    key_to_mutate = np.random.choice(['num_layers', 'neurons_per_layer', 'activation_functions', 'optimizer', 'alpha', 'loss_function'])
    if key_to_mutate == 'num_layers':
        chromosome[key_to_mutate] = np.random.choice(search_space['num_layers'])
        chromosome['neurons_per_layer'] = np.random.choice(search_space['neurons_per_layer'], chromosome[key_to_mutate]).tolist()
        chromosome['activation_functions'] = np.random.choice(search_space['activation_functions'], chromosome[key_to_mutate]).tolist()
    elif key_to_mutate == 'neurons_per_layer':
        layer_to_mutate = np.random.randint(0, chromosome['num_layers'])
        chromosome[key_to_mutate][layer_to_mutate] = np.random.choice(search_space['neurons_per_layer'])
    elif key_to_mutate == 'activation_functions':
        layer_to_mutate = np.random.randint(0, chromosome['num_layers'])
        chromosome[key_to_mutate][layer_to_mutate] = np.random.choice(search_space['activation_functions'])
    elif key_to_mutate == 'optimizer':
        chromosome[key_to_mutate] = np.random.choice(search_space['optimizers'])
    elif key_to_mutate == 'alpha':
        chromosome[key_to_mutate] = np.random.choice(search_space['alphas'])
    elif key_to_mutate == 'loss_function':
        chromosome[key_to_mutate] = np.random.choice(search_space['loss_functions'])
    # Adjust the lists to the number of layers
    chromosome['neurons_per_layer'] = chromosome['neurons_per_layer'][:chromosome['num_layers']]
    chromosome['activation_functions'] = chromosome['activation_functions'][:chromosome['num_layers']]
    return chromosome

def select_parents(population, num_parents):
    sorted_population = sorted(population, key=lambda x: x['fitness'], reverse=True)
    return sorted_population[:num_parents]

# Define the genetic algorithm parameters
population_size = 4
num_generations = 2
num_parents_to_select = 2

# Initialize the population
population = initialize_population(population_size)

# Store parameters and R2 scores on each iteration
results = []

# Main loop of the genetic algorithm
for generation in range(num_generations):
    # Evaluate the fitness of each individual in the population
    for individual in population:
        individual['fitness'] = evaluate_model(individual)
    
    # Store parameters and R2 scores
    for individual in population:
        result = {
            'num_layers': individual['num_layers'],
            'neurons_per_layer': individual['neurons_per_layer'],
            'activation_functions': individual['activation_functions'],
            'optimizer': individual['optimizer'],
            'alpha': individual['alpha'],
            'loss_function': individual['loss_function'],
            'R2_score': individual['fitness']
        }
        results.append(result)
    
    # Select the best individuals as parents for the next generation
    parents = select_parents(population, num_parents_to_select)
    
    # Generate offspring through crossover and mutation
    offspring = []
    while len(offspring) < population_size:
        parent1 = np.random.choice(parents)
        parent2 = np.random.choice(parents)
        child = crossover(parent1, parent2)
        if np.random.rand() < 0.1:  # Mutation probability
            child = mutate(child)
        offspring.append(child)
    
    # Replace the old population with the offspring
    population = offspring

# Store results in a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv('genetic_algorithm_results_5.csv', index=False)


18/18 [==============================] - 0s 2ms/step


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
import pandas as pd
import os

# Load the dataset
data = pd.read_csv('water_potability.csv')

# Calculate the count of missing values for each column
missing_counts = data.isnull().sum()

# Select the indices where the missing counts are zero
zero_missing_indices = missing_counts.index[missing_counts.values == 0]

# Select the indices where the missing counts are non-zero
non_zero_missing_indices = missing_counts.index[missing_counts.values > 0]

# Create a new dataset with only the columns having no missing values
data_filled = data[zero_missing_indices]

# Create a new dataset with only the columns having missing values
data_skipped = data[non_zero_missing_indices]

# Add the first column from data_skipped to data_filled
data1 = pd.concat([data_filled, data_skipped.iloc[:, 0:1]], axis=1)

# Select rows from data1 where the 'ph' column is not NaN
data1_filled = data1[data1['ph'].notna()]

# Split data into features and target
X = data1_filled.drop(['ph'], axis=1)
y = data1_filled['ph']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the search space for hyperparameters
search_space = {
    'num_layers': np.arange(1, 20),  # Number of layers
    'neurons_per_layer': np.arange(1, 128, 1),  # Neurons per layer
    'activation_functions': ['relu', 'sigmoid', 'tanh', 'softmax', 'softplus',
                             'softsign', 'elu', 'selu', 'gelu', 'hard_sigmoid', 'linear'],  # Activation functions
    'optimizers': ['adam', 'sgd', 'rmsprop', 'adagrad', 'adadelta', 'adamax', 'nadam'],  # Optimizers
    'alphas': [0.0001, 0.001, 0.01, 0.1],  # Learning rates
    'loss_functions': ['mean_squared_error', 'mean_absolute_error',  'logcosh', 'huber_loss']
}

# Function to get optimizer object
def get_optimizer(name, alpha):
    optimizers = {
        'adam': Adam(learning_rate=alpha),
        'sgd': SGD(learning_rate=alpha),
        'rmsprop': RMSprop(learning_rate=alpha),
        'adagrad': Adagrad(learning_rate=alpha),
        'adadelta': Adadelta(learning_rate=alpha),
        'adamax': Adamax(learning_rate=alpha),
        'nadam': Nadam(learning_rate=alpha)
    }
    return optimizers[name]

# Define the fitness function
def evaluate_model(params, individual_id):
    num_layers = params['num_layers']
    neurons_per_layer = params['neurons_per_layer']
    activation_functions = params['activation_functions']
    optimizer_name = params['optimizer']
    alpha = params['alpha']
    loss_function = params['loss_function']
    
    model = Sequential()
    model.add(Dense(neurons_per_layer[0], activation=activation_functions[0], input_shape=(X_train.shape[1],)))
    for i in range(1, num_layers):
        model.add(Dense(neurons_per_layer[i], activation=activation_functions[i]))
    model.add(Dense(1))  # Output layer
    optimizer = get_optimizer(optimizer_name, alpha)
    model.compile(optimizer=optimizer, loss=loss_function)
    
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    
    # Save parameters to a .txt file
    with open(f'individuals/individual_{individual_id}_params.txt', 'w') as file:
        file.write(f'num_layers: {num_layers}\n')
        file.write(f'neurons_per_layer: {neurons_per_layer}\n')
        file.write(f'activation_functions: {activation_functions}\n')
        file.write(f'optimizer: {optimizer_name}\n')
        file.write(f'alpha: {alpha}\n')
        file.write(f'loss_function: {loss_function}\n')
        file.write(f'r2_score: {r2}\n')
        
    # Save loss history to a .txt file
    with open(f'individuals/individual_{individual_id}_loss_history.txt', 'w') as file:
        for loss in history.history['loss']:
            file.write(f'{loss}\n')
    
    # Save y_test and y_pred to a .txt file
    predictions = predictions.flatten()
    with open(f'individuals/individual_{individual_id}_predictions.txt', 'w') as file:
        for true, pred in zip(y_test, predictions):
            file.write(f'{true}, {pred}\n')
    
    return r2

# Define the genetic algorithm operations
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        num_layers = np.random.choice(search_space['num_layers'])
        neurons_per_layer = np.random.choice(search_space['neurons_per_layer'], num_layers).tolist()
        activation_functions = np.random.choice(search_space['activation_functions'], num_layers).tolist()
        optimizer = np.random.choice(search_space['optimizers'])
        alpha = np.random.choice(search_space['alphas'])
        loss_function = np.random.choice(search_space['loss_functions'])
        chromosome = {
            'num_layers': num_layers,
            'neurons_per_layer': neurons_per_layer,
            'activation_functions': activation_functions,
            'optimizer': optimizer,
            'alpha': alpha,
            'loss_function': loss_function
        }
        population.append(chromosome)
    return population

def crossover(parent1, parent2):
    child = {}
    child['num_layers'] = np.random.choice([parent1['num_layers'], parent2['num_layers']])
    child['neurons_per_layer'] = [np.random.choice([p1, p2]) for p1, p2 in zip(parent1['neurons_per_layer'], parent2['neurons_per_layer'])]
    child['activation_functions'] = [np.random.choice([p1, p2]) for p1, p2 in zip(parent1['activation_functions'], parent2['activation_functions'])]
    child['optimizer'] = np.random.choice([parent1['optimizer'], parent2['optimizer']])
    child['alpha'] = np.random.choice([parent1['alpha'], parent2['alpha']])
    child['loss_function'] = np.random.choice([parent1['loss_function'], parent2['loss_function']])
    # Adjust the lists to the number of layers
    child['neurons_per_layer'] = child['neurons_per_layer'][:child['num_layers']]
    child['activation_functions'] = child['activation_functions'][:child['num_layers']]
    return child

def mutate(chromosome):
    key_to_mutate = np.random.choice(['num_layers', 'neurons_per_layer', 'activation_functions', 'optimizer', 'alpha', 'loss_function'])
    if key_to_mutate == 'num_layers':
        chromosome[key_to_mutate] = np.random.choice(search_space['num_layers'])
        chromosome['neurons_per_layer'] = np.random.choice(search_space['neurons_per_layer'], chromosome[key_to_mutate]).tolist()
        chromosome['activation_functions'] = np.random.choice(search_space['activation_functions'], chromosome[key_to_mutate]).tolist()
    elif key_to_mutate == 'neurons_per_layer':
        layer_to_mutate = np.random.randint(0, chromosome['num_layers'])
        chromosome[key_to_mutate][layer_to_mutate] = np.random.choice(search_space['neurons_per_layer'])
    elif key_to_mutate == 'activation_functions':
        layer_to_mutate = np.random.randint(0, chromosome['num_layers'])
        chromosome[key_to_mutate][layer_to_mutate] = np.random.choice(search_space['activation_functions'])
    elif key_to_mutate == 'optimizer':
        chromosome[key_to_mutate] = np.random.choice(search_space['optimizers'])
    elif key_to_mutate == 'alpha':
        chromosome[key_to_mutate] = np.random.choice(search_space['alphas'])
    elif key_to_mutate == 'loss_function':
        chromosome[key_to_mutate] = np.random.choice(search_space['loss_functions'])
    # Adjust the lists to the number of layers
    chromosome['neurons_per_layer'] = chromosome['neurons_per_layer'][:chromosome['num_layers']]
    chromosome['activation_functions'] = chromosome['activation_functions'][:chromosome['num_layers']]
    return chromosome

def select_parents(population, num_parents):
    sorted_population = sorted(population, key=lambda x: x['fitness'], reverse=True)
    return sorted_population[:num_parents]

# Define the genetic algorithm parameters
population_size = 100
num_generations = 4
num_parents_to_select = 10

# Initialize the population
population = initialize_population(population_size)

# Store parameters and R2 scores on each iteration
results = []

# Main loop of the genetic algorithm
for generation in range(num_generations):
    # Evaluate the fitness of each individual in the population
    for i, individual in enumerate(population):
        individual['fitness'] = evaluate_model(individual, f'g{generation}_i{i}')
    
    # Store parameters and R2 scores
    for individual in population:
        result = {
            'num_layers': individual['num_layers'],
            'neurons_per_layer': individual['neurons_per_layer'],
            'activation_functions': individual['activation_functions'],
            'optimizer': individual['optimizer'],
            'alpha': individual['alpha'],
            'loss_function': individual['loss_function'],
            'R2_score': individual['fitness']
        }
        results.append(result)
    
    # Select the best individuals as parents for the next generation
    parents = select_parents(population, num_parents_to_select)
    
    # Generate offspring through crossover and mutation
    offspring = []
    while len(offspring) < population_size:
        parent1 = np.random.choice(parents)
        parent2 = np.random.choice(parents)
        child = crossover(parent1, parent2)
        if np.random.rand() < 0.1:  # Mutation probability
            child = mutate(child)
        offspring.append(child)
    
    # Replace the old population with the offspring
    population = offspring

# Store results in a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv('genetic_algorithm_results_6.csv', index=False)


18/18 [==============================] - 0s 3ms/step



KeyboardInterrupt



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import roc_auc_score, r2_score

# Load the dataset
data = pd.read_csv('water_potability.csv')

# Calculate the count of missing values for each column
missing_counts = data.isnull().sum()

# Select the indices where the missing counts are zero
zero_missing_indices = missing_counts.index[missing_counts.values == 0]

# Select the indices where the missing counts are non-zero
non_zero_missing_indices = missing_counts.index[missing_counts.values > 0]

# Create a new dataset with only the columns having no missing values
data_filled = data[zero_missing_indices]

# Add the first column from data_skipped to data_filled
data1 = pd.concat([data_filled, data_skipped.iloc[:, 0:1]], axis=1)

# Select rows from data1 where the 'ph' column is not NaN
data1_filled = data1[data1['ph'].notna()]

# Split data into features and target
X = data1_filled.drop(['ph'], axis=1)
y = data1_filled['ph']
# Split data into features and target
X = data1_filled.drop(['ph'], axis=1)
y = data1_filled['ph']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(18, activation='gelu', input_shape=(X_train.shape[1],)))
model.add(Dense(97, activation='selu'))
model.add(Dense(47, activation='softplus'))
model.add(Dense(39, activation='relu'))
model.add(Dense(54, activation='softplus'))
model.add(Dense(9, activation='softsign'))
model.add(Dense(55, activation='gelu'))
model.add(Dense(104, activation='sigmoid'))
model.add(Dense(42, activation='softsign'))
model.add(Dense(37, activation='sigmoid'))
model.add(Dense(1))
model.compile(optimizer=Adadelta(lr=0.01), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# Evaluate the model
y_pred = model.predict(X_test)


# Calculate R2 score
r2 = r2_score(y_test, y_pred)

print("R2 Score:", r2)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os

# Load the dataset
data = pd.read_csv('water_potability.csv')

# Calculate the count of missing values for each column
missing_counts = data.isnull().sum()

# Select the indices where the missing counts are zero
zero_missing_indices = missing_counts.index[missing_counts.values == 0]

# Select the indices where the missing counts are non-zero
non_zero_missing_indices = missing_counts.index[missing_counts.values > 0]

# Create a new dataset with only the columns having no missing values
data_filled = data[zero_missing_indices]

# Create a new dataset with only the columns having missing values
data_skipped = data[non_zero_missing_indices]

# Add the first column from data_skipped to data_filled
data1 = pd.concat([data_filled, data_skipped.iloc[:, 0:1]], axis=1)

# Select rows from data1 where the 'ph' column is not NaN
data1_filled = data1[data1['ph'].notna()]

# Split data into features and target
X = data1_filled.drop(['ph'], axis=1)
y = data1_filled['ph']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the search space for hyperparameters
search_space = {
    'num_layers': np.arange(1, 20),  # Number of layers
    'neurons_per_layer': np.arange(1, 128, 1),  # Neurons per layer
    'activation_functions': ['relu', 'sigmoid', 'tanh', 'softmax', 'softplus',
                             'softsign', 'elu', 'selu', 'gelu', 'hard_sigmoid', 'linear'],  # Activation functions
    'optimizers': ['adam', 'sgd', 'rmsprop', 'adagrad', 'adadelta', 'adamax', 'nadam'],  # Optimizers
    'alphas': [0.0001, 0.001, 0.01, 0.1],  # Learning rates
    'loss_functions': ['mean_squared_error', 'mean_absolute_error',  'logcosh', 'huber_loss']
}

# Function to get optimizer object
def get_optimizer(name, alpha):
    optimizers = {
        'adam': Adam(learning_rate=alpha),
        'sgd': SGD(learning_rate=alpha),
        'rmsprop': RMSprop(learning_rate=alpha),
        'adagrad': Adagrad(learning_rate=alpha),
        'adadelta': Adadelta(learning_rate=alpha),
        'adamax': Adamax(learning_rate=alpha),
        'nadam': Nadam(learning_rate=alpha)
    }
    return optimizers[name]

# Define the fitness function
def evaluate_model(params, individual_id):
    num_layers = params['num_layers']
    neurons_per_layer = params['neurons_per_layer']
    activation_functions = params['activation_functions']
    optimizer_name = params['optimizer']
    alpha = params['alpha']
    loss_function = params['loss_function']
    
    model = Sequential()
    model.add(Dense(neurons_per_layer[0], activation=activation_functions[0], input_shape=(X_train.shape[1],)))
    for i in range(1, num_layers):
        model.add(Dense(neurons_per_layer[i], activation=activation_functions[i]))
    model.add(Dense(1))  # Output layer
    optimizer = get_optimizer(optimizer_name, alpha)
    model.compile(optimizer=optimizer, loss=loss_function)
    
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    
    # Save parameters to a .txt file
    with open(f'individuals/individual_{individual_id}_params.txt', 'w') as file:
        file.write(f'num_layers: {num_layers}\n')
        file.write(f'neurons_per_layer: {neurons_per_layer}\n')
        file.write(f'activation_functions: {activation_functions}\n')
        file.write(f'optimizer: {optimizer_name}\n')
        file.write(f'alpha: {alpha}\n')
        file.write(f'loss_function: {loss_function}\n')
        file.write(f'r2_score: {r2}\n')
    
    # Save loss history to a .txt file
    with open(f'individuals/individual_{individual_id}_loss_history.txt', 'w') as file:
        for loss in history.history['loss']:
            file.write(f'{loss}\n')
    
    # Save y_test and y_pred to a .txt file
    with open(f'individuals/individual_{individual_id}_predictions.txt', 'w') as file:
        for index, true, pred in zip(list(y_test.index), y_test, predictions):
            file.write(f'{index}, {true}, {pred[0]}\n')
    
    return r2

# Define the genetic algorithm operations
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        num_layers = np.random.choice(search_space['num_layers'])
        neurons_per_layer = np.random.choice(search_space['neurons_per_layer'], num_layers).tolist()
        activation_functions = np.random.choice(search_space['activation_functions'], num_layers).tolist()
        optimizer = np.random.choice(search_space['optimizers'])
        alpha = np.random.choice(search_space['alphas'])
        loss_function = np.random.choice(search_space['loss_functions'])
        chromosome = {
            'num_layers': num_layers,
            'neurons_per_layer': neurons_per_layer,
            'activation_functions': activation_functions,
            'optimizer': optimizer,
            'alpha': alpha,
            'loss_function': loss_function
        }
        population.append(chromosome)
    return population

def crossover(parent1, parent2):
    child = {}
    child['num_layers'] = np.random.choice([parent1['num_layers'], parent2['num_layers']])
    child['neurons_per_layer'] = [np.random.choice([p1, p2]) for p1, p2 in zip(parent1['neurons_per_layer'], parent2['neurons_per_layer'])]
    child['activation_functions'] = [np.random.choice([p1, p2]) for p1, p2 in zip(parent1['activation_functions'], parent2['activation_functions'])]
    child['optimizer'] = np.random.choice([parent1['optimizer'], parent2['optimizer']])
    child['alpha'] = np.random.choice([parent1['alpha'], parent2['alpha']])
    child['loss_function'] = np.random.choice([parent1['loss_function'], parent2['loss_function']])
    # Adjust the lists to the number of layers
    child['neurons_per_layer'] = child['neurons_per_layer'][:child['num_layers']]
    child['activation_functions'] = child['activation_functions'][:child['num_layers']]
    return child

def mutate(chromosome):
    key_to_mutate = np.random.choice(['num_layers', 'neurons_per_layer', 'activation_functions', 'optimizer', 'alpha', 'loss_function'])
    if key_to_mutate == 'num_layers':
        chromosome[key_to_mutate] = np.random.choice(search_space['num_layers'])
        chromosome['neurons_per_layer'] = np.random.choice(search_space['neurons_per_layer'], chromosome[key_to_mutate]).tolist()
        chromosome['activation_functions'] = np.random.choice(search_space['activation_functions'], chromosome[key_to_mutate]).tolist()
    elif key_to_mutate == 'neurons_per_layer':
        if chromosome['num_layers'] > 0:
            layer_to_mutate = np.random.randint(0, chromosome['num_layers'])
            chromosome[key_to_mutate][layer_to_mutate] = np.random.choice(search_space['neurons_per_layer'])
    elif key_to_mutate == 'activation_functions':
        if chromosome['num_layers'] > 0:
            layer_to_mutate = np.random.randint(0, chromosome['num_layers'])
            chromosome[key_to_mutate][layer_to_mutate] = np.random.choice(search_space['activation_functions'])
    elif key_to_mutate == 'optimizer':
        chromosome[key_to_mutate] = np.random.choice(search_space['optimizers'])
    elif key_to_mutate == 'alpha':
        chromosome[key_to_mutate] = np.random.choice(search_space['alphas'])
    elif key_to_mutate == 'loss_function':
        chromosome[key_to_mutate] = np.random.choice(search_space['loss_functions'])
    
    # Adjust the lists to the number of layers
    chromosome['neurons_per_layer'] = chromosome['neurons_per_layer'][:chromosome['num_layers']]
    chromosome['activation_functions'] = chromosome['activation_functions'][:chromosome['num_layers']]
    
    return chromosome



def select_parents(population, num_parents):
    sorted_population = sorted(population, key=lambda x: x['fitness'], reverse=True)
    return sorted_population[:num_parents]

# Define the genetic algorithm parameters
population_size = 3
num_generations = 3
num_parents_to_select = 2

# Initialize the population
population = initialize_population(population_size)

# Store parameters and R2 scores on each iteration
results = []

# Main loop of the genetic algorithm
for generation in range(num_generations):
    # Evaluate the fitness of each individual in the population
    for i, individual in enumerate(population):
        individual['fitness'] = evaluate_model(individual, f'g{generation}_i{i}')
    
    # Store parameters and R2 scores
    for individual in population:
        result = {
            'num_layers': individual['num_layers'],
            'neurons_per_layer': individual['neurons_per_layer'],
            'activation_functions': individual['activation_functions'],
            'optimizer': individual['optimizer'],
            'alpha': individual['alpha'],
            'loss_function': individual['loss_function'],
            'R2_score': individual['fitness']
        }
        results.append(result)
    
    # Select the best individuals as parents for the next generation
    parents = select_parents(population, num_parents_to_select)
    
    # Generate offspring through crossover and mutation
    offspring = []
    while len(offspring) < population_size:
        parent1 = np.random.choice(parents)
        parent2 = np.random.choice(parents)
        child = crossover(parent1, parent2)
        if np.random.rand() < 0.1:  # Mutation probability
            child = mutate(child)
        offspring.append(child)
    
    # Replace the old population with the offspring
    population = offspring

# Store results in a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv('genetic_algorithm_results_6.csv', index=False)
